In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
from pyfaidx import Fasta
import upsetplot
from pandarallel import pandarallel

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [44]:
mt_df = pd.read_csv('../data/05_mastertable/poder_master_table_fixed_genics.tsv', sep='\t')
# mt_df = mt_df.loc[mt_df['filter']=='pass']
mt_df['pop_spec_t'] = (mt_df.population_sharing==1)&\
                      (mt_df.sample_sharing>1)


In [46]:
len(mt_df)

155875

In [47]:
meta_df = load_meta()
samples = meta_df['sample'].unique().tolist()
samples = [c for c in samples if c in mt_df.columns]

In [48]:
keep_cols = ['isoform', 'structural_category']+samples
keep_cols = [c for c in keep_cols if c in mt_df.columns]

In [49]:
mt_df = mt_df[keep_cols].melt(id_vars=['isoform', 'structural_category'], value_vars=samples)

In [51]:
print(len(mt_df.index))
mt_df = mt_df.loc[mt_df.value==1]
print(len(mt_df.index))

6702625
2048259


In [52]:
# only noel
print(len(mt_df.index))
mt_df = mt_df.loc[mt_df.structural_category!='FSM']
print(len(mt_df.index))

2048259
524750


In [53]:
mt_df = mt_df[['isoform', 'variable']].groupby('variable').nunique().reset_index().rename({'isoform':'n_novel_iso'}, axis=1)


In [54]:
mt_df.n_novel_iso.median()

11700.0

In [55]:
mt_df.head()

,variable,n_novel_iso
0,AJI1,8468
1,AJI2,11517
2,AJI3,11942
3,AJI4,9396
4,AJI5,13256


In [56]:
sns.distplot(mt_df, y='n_novel_iso')

TypeError: distplot() got an unexpected keyword argument 'y'